In [2]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import time
import imageio
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [3]:
data_path = os.path.join(Path(os.getcwd()).parent.absolute(), "Data")
# video_path = os.path.join(data_path, "Suitcase.mp4")
video_path = os.path.join(data_path, "cut.mp4")
# video_path = os.path.join(data_path, "cut3.mp4")
vidcap = cv2.VideoCapture(video_path)
success, image = vidcap.read()

# # get two frames of the video
# count = 0
# while success and count < 53:
#     if count > 50:
#         cv2.imwrite("frame%d.jpg" % count, image) 
#         print('Read a new frame', count)
#     success, image = vidcap.read()
#     count += 1

In [4]:
# Video cut
# ffmpeg_extract_subclip(video_path, 10, 15, targetname=os.path.join(data_path,"cut3.mp4"))

In [5]:
def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr


def draw_hsv(flow):

    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

In [8]:
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

img_lst = []
img_lst2 = []

while True:

    suc, img = cap.read()

    if img is None :
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # start time to calculate FPS
    start = time.time()

    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    prevgray = gray

    # End time
    end = time.time()

    # calculate the FPS for current frame detection
    fps = 1 / (end-start)
    # print(f"{fps:.2f} FPS")

    img_lst.append(draw_flow(gray, flow))
    img_lst2.append(draw_hsv(flow))

    cv2.imshow('flow', draw_flow(gray, flow))
    cv2.imshow('flow HSV', draw_hsv(flow))

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

imageio.mimsave(os.path.join(data_path, "flow.gif"), img_lst)
imageio.mimsave(os.path.join(data_path, "flow_hsv.gif"), img_lst2)